In [1]:
import sparknlp

# spark = sparknlp.start()

from sparknlp.base import *
from sparknlp.annotator import *
from transformers import T5Tokenizer, T5ForConditionalGeneration
print("Spark NLP version", sparknlp.version())
# print("Apache Spark version:", spark.version)

# spark

/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Spark NLP version 5.3.2


/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
from pyspark.ml import Pipeline
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StringType, IntegerType

In [3]:
spark = SparkSession.builder.appName("Spark NLP") \
    .config(
        "spark.jars", 
        "./kafka-clients-3.5.0.jar,./spark-sql-kafka-0-10_2.12-3.5.0.jar, \
        ./spark-token-provider-kafka-0-10_2.12-3.5.0.jar, \
        ./commons-pool2-2.12.0.jar") \
    .config("spark.sql.shuffle.partitions", "3") \
    .config("spark.executorEnv.PYSPARK_PYTHON","/root/anaconda3/bin/python") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.log.level", "ERROR") \
    .master("local[*]") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e802197c-3234-49db-9fdb-6aa0c26bd78d;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.g

In [10]:
# !wget -q -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

In [4]:
import numpy as np
import pandas as pd

In [5]:
p2_df = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "p2") \
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
p2_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
p2_df.writeStream \
    .format("memory") \
    .outputMode("append") \
    .queryName("validation_set") \
    .option("checkpointLocation", "./checkpoint") \
    .start()

In [9]:
df = spark.sql("select CAST(value AS STRING) from validation_set")

In [10]:
schema = "id STRING, title STRING, context STRING, question STRING, answers STRUCT<text: ARRAY<STRING>, answer_start: ARRAY<INT>>"
parsed_df = df \
    .withColumn("parsed_value", from_json(col("value"), schema)) \
    .select("parsed_value.*")

parsed_df.show(5, truncate=False)

+------------------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|id             

In [22]:
import transformers

MODEL_NAME = "output/debug_seq2seq_squad/checkpoint-500"

EXPORT_PATH = f"output/onnx/flan_t5_finetuned"

In [23]:
!optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}

/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/3: T5Stack *****
Using framework PyTorch: 2.3.0+cu121
Overriding 1 c

In [31]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

local_model_path = "./output/onnx/flan_t5_finetuned"

t5 = T5Transformer \
    .loadSavedModel(local_model_path, spark) \
    .setUseCache(True) \
    .setInputCols(["documents"]) \
    .setOutputCol("outputs")

pipeline = Pipeline().setStages([document_assembler, t5])


Using CPUs
Using CPUs


In [27]:
import pyspark.sql.functions as F

parsed_df = parsed_df.withColumn('text', F.format_string('question: %s context: %s', F.col('question'), F.col('context')))

results = pipeline.fit(parsed_df).transform(parsed_df)

In [30]:
results.select("question", "outputs").sample(0.3).show(5, truncate = False)

+-----------------------------------------------------------------------+--------------------------------------------------------------------------+
|question                                                               |outputs                                                                   |
+-----------------------------------------------------------------------+--------------------------------------------------------------------------+
|In what country is Normandy located?                                   |[{document, 0, 5, France, {sentence -> 0}, []}]                           |
|From which countries did the Norse originate?                          |[{document, 0, 26, Denmark, Iceland and Norway, {sentence -> 0}, []}]     |
|What century did the Normans first gain their separate identity?       |[{document, 0, 11, 10th century, {sentence -> 0}, []}]                    |
|What is France a region of?                                            |[{document, 0, 7, Normandy, {sent